In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S5'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 5 #Change from 3 to 5 as in paper
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 3004
Total Tokens Found = 38106
Total Tokens Output = 38026
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 3004
Total Tokens= 38026
Unique Tokens= 7190
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  5
Maximum Frequency of Error Token =  3
References Processed= 3004
Sorted Token Size = 7190
Clean Token Size = 2887
*Stop Replacements here
References Processed =  3004
Total Replacement Pairs = 148
Tokens Read = 38026
Tokens Changed =  216
References Changed = 210


In [2]:
status="done"

In [3]:
print(status)

done


In [4]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [5]:
import numpy as np

Create a function of the parameters that returns the F score

In [6]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [7]:
entropy_F1(mu=0.5,muIncr=0.1,beta=12,sigma=53,epsilon=3.0,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 3567
NewMatrix Iterateblocks Total Time = 1.613547770306468


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2642
NewMatrix Iterateblocks Total Time = 1.4009002530947328


>>>>>>>>>>>>
Starting Iteration mu= 0.7
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1632
NewMatrix Iterateblocks Total Time = 1.0055415872484446


>>>>>>>>>>>>
Starting Iteration mu= 0.7999999999999999
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 842
NewMatrix Iterateblocks Total Time = 0.7899485100060701


>>>>>>>>>>>>
Starting Iteration mu= 0.8999999999999999
Starting Iteration epsilon= 3.0
Running NewMatrix Jar!
 Single-Reference Bl

0.91684285

Create functions for variable transformation.

In [8]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [9]:
inv_softplus(1)

0.541324854612918

In [10]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [11]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [12]:
import os, sys

In [13]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [14]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [15]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [17]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-0.2456525596239042

In [16]:
optimizer=None

In [17]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [18]:
from bayes_opt import SequentialDomainReductionTransformer

In [19]:
bounds_transformer = SequentialDomainReductionTransformer()

In [20]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
    bounds_transformer=bounds_transformer
)

In [21]:
# optimizer.probe(
#     params={"beta_sp": inv_softplus(12), 
#             "sigma_sp": inv_softplus(53),
#             "mu_logit":logit(0.5),
#             #'epsilonIterate_logit': logit(0),
#             "muIncr_logit":logit(0.1),
#             "epsilon_sp":inv_softplus(3.0)},
#     lazy=True,
# )

In [22]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
    acq="ucb",
    kappa=5,
    alpha=1
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  1        |  1.886    |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  2        |  1.709    |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  3        |  0.7837   |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  4        | -1.7      |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|EpIter:0.00
|  5        |  1.333    |  560.6    |  677.8    | -3.589    |  3.181    |  613.5    |
Mu:0.69|Mu+:0.01|Beta:626.00|Sigma:614.00|Epsilon:60.03

|  39       |  1.897    |  308.2    |  641.3    | -3.675    |  1.359    |  96.95    |
Mu:0.80|Mu+:0.03|Beta:318.00|Sigma:86.00|Epsilon:631.35|EpIter:0.00
|  40       |  1.861    |  318.9    |  631.3    | -3.633    |  1.355    |  86.57    |
Mu:0.79|Mu+:0.02|Beta:309.00|Sigma:95.00|Epsilon:640.31|EpIter:0.00
|  41       |  1.907    |  309.3    |  640.3    | -3.671    |  1.352    |  95.92    |
Mu:0.79|Mu+:0.02|Beta:317.00|Sigma:87.00|Epsilon:632.24|EpIter:0.00
|  42       |  1.9      |  317.9    |  632.2    | -3.669    |  1.349    |  87.51    |
Mu:0.79|Mu+:0.02|Beta:310.00|Sigma:95.00|Epsilon:639.51|EpIter:0.00
|  43       |  1.914    |  310.1    |  639.5    | -3.668    |  1.346    |  95.07    |
Mu:0.79|Mu+:0.02|Beta:317.00|Sigma:94.00|Epsilon:632.97|EpIter:0.00
|  44       |  1.925    |  317.1    |  633.0    | -3.666    |  1.344    |  94.72    |
Mu:0.79|Mu+:0.02|Beta:310.00|Sigma:88.00|Epsilon:638.85|EpIter:0.00
|  45       |  1.912    |  310.8    |  638.9    | -3.665    |  1.342    |  8

In [23]:
optimizer.max

{'target': 1.9331217685462174,
 'params': {'beta_sp': 313.8243372798073,
  'epsilon_sp': 636.0663316911659,
  'muIncr_logit': -3.6529632868977835,
  'mu_logit': 1.3223621707173336,
  'sigma_sp': 91.49093763950985}}

In [24]:
inv_logit(optimizer.max["target"])

0.87359455

In [25]:
params=optimizer.max["params"]

In [26]:
softplus(params["beta_sp"])

313.8243372798073

In [27]:
softplus(params["sigma_sp"])

91.49093763950985

In [28]:
inv_logit(params["mu_logit"])

0.7895744417142209

In [29]:
inv_logit(params["muIncr_logit"])

0.025259639659271495

In [30]:
softplus(params["epsilon_sp"])

636.0663316911659

In [31]:
entropy_F1(mu=0.789,
           muIncr=0.0253,
           beta=313,
           sigma=91,
           epsilon=636.07,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.789
Starting Iteration epsilon= 636.07
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 3885
NewMatrix Iterateblocks Total Time = 10.76923129055649

Precision = TP / (TP + FP) ......................... = 0.90992908
Recall = TP / (TP + FN) ............................ = 0.84076016
F-Measure .......................................... = 0.87397820


0.8739782